In [1]:
import os, math, glob
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np
from transformers import AutoProcessor, AutoModel
import networkx as nx
import cv2
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from sklearn.decomposition import PCA
import faiss
import pycolmap
import os
import sqlite3
from collections import defaultdict
from scipy.spatial import cKDTree
import h5py
from pathlib import Path

"""
1) Global feature matching using DINOV2
- to reduce error propagation use PCA-whitening :why? -> essentially to remove redundent patterns, and highlight more meaningful patterns
- use FAISS HNSW and query top-K neighbors, Faiss for similarity search, finding similarity based on THOSE meaningful patterns. and query top-k neighbours amongst the candidates from similarity search

- https://www.researchgate.net/publication/259624676_Negative_Evidences_and_Co-occurences_in_Image_Retrieval_The_Benefit_of_PCA_and_Whitening
2) use the image pairings from DINOV2 
3) Within each image pairings do:
    1) set longside to 960px and max keypoinys to 4096,superpoint for extraction then LightGlue (feature matching)
    2) insert pair into colmap to do geometric verification using RANSAC or do manual: Do Ransaac to do geometric verification of each match (remove outliers through sampson approx -> https://arxiv.org/abs/2401.07114)
4) pycolmap
"""



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
global_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14').to(device).eval()
preprocess = transforms.Compose([
    transforms.Resize(518, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(518),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225)),
])
local_processor = AutoProcessor.from_pretrained("ETH-CVG/lightglue_superpoint",use_fast=True)
local_model = AutoModel.from_pretrained("ETH-CVG/lightglue_superpoint").to(device).eval()

/mnt/c/Users/asyra/Projects/feature_matching_kag/.pyenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /home/asy/.cache/torch/hub/facebookresearch_dinov2_main
/home/asy/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/asy/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/asy/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
`use_fast` is set to `True` but the image processor class does not have a fast version.  

In [2]:


@torch.no_grad()
def dino_forward(images_tensor):
    with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
        out = global_model.forward_features(images_tensor)
    feats = (out["x_norm_clstoken"]
             if isinstance(out, dict) and out.get("x_norm_clstoken") is not None
             else out["x_norm_patchtokens"].mean(dim=1))
    feats = torch.nn.functional.normalize(feats, dim=1)
    return feats.float()

def load_pil(path):
    im = Image.open(path).convert("RGB")
    return preprocess(im)

def extract_embeddings(folder, batch_size=32):
    exts = ("*.jpg","*.jpeg","*.png")
    files = []
    for e in exts:
        files += glob.glob(os.path.join(folder, e))

    X = []
    batch = []
    paths = []
    for fp in files:
        try:
            batch.append(load_pil(fp))
            paths.append(fp)
            if len(batch) == batch_size:
                imgs = torch.stack(batch).to(device)
                X.append(dino_forward(imgs).cpu().numpy())
                batch.clear()
        except Exception as e:
            print(f"[WARN] skip {fp}: {e}")
    if batch:
        imgs = torch.stack(batch).to(device)
        X.append(dino_forward(imgs).cpu().numpy())

    X = np.vstack(X).astype("float32")
    X /= np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
    return paths, X

def pca_whiten(X, out_dim=256, seed=42, whiten=True):
    # X: (N, D)
    N, D = X.shape
    max_dim = max(1, min(N - 1, D))
    k = min(out_dim, max_dim)
    if k < out_dim:
        print(f"[PCA] Reducing out_dim from {out_dim} -> {k} (N={N}, D={D})")
    pca = PCA(n_components=k, svd_solver="auto", random_state=seed, whiten=whiten)
    Xw = pca.fit_transform(X).astype("float32")
    Xw /= np.linalg.norm(Xw, axis=1, keepdims=True) + 1e-12
    return Xw, pca

def ann_search(X, K=40):
    d = X.shape[1]
    idx = faiss.IndexHNSWFlat(d, 32)
    idx.hnsw.efConstruction = 300
    idx.add(X)
    idx.hnsw.efSearch = 256
    D, I = idx.search(X, K+1)
    return I[:, 1:], D[:, 1:]

def mutual_knn_pairs(I):
    N, K = I.shape
    neigh = [set(row.tolist()) for row in I]
    pairs = set()
    for i in range(N):
        for j in I[i]:
            if i < j and i in neigh[j]:
                pairs.add((i, j))
    return pairs

def jaccard_prune(pairs, I, jaccard_min=0.25):
    neigh = [set(row.tolist()) for row in I]
    keep = []
    for i,j in pairs:
        inter = len(neigh[i] & neigh[j])
        union = len(neigh[i] | neigh[j])
        jac = inter / (union + 1e-9)
        if jac >= jaccard_min:
            keep.append((i,j))
    return keep

def global_image_pairing(folder,
                         batch_size=32,
                         K=40,
                         use_pca=True,
                         pca_dim=256,
                         jaccard_min=0.25):
    paths, X = extract_embeddings(folder, batch_size=batch_size)
    if use_pca:
        X, _ = pca_whiten(X, out_dim=pca_dim)
    I, _ = ann_search(X, K=K)
    mk = mutual_knn_pairs(I)
    pairs = jaccard_prune(mk, I, jaccard_min=jaccard_min)
    path_pairs = [(paths[i], paths[j]) for (i,j) in pairs]
    print(path_pairs)
    return paths, X, path_pairs

def image_size(path):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None:
        raise FileNotFoundError(path)
    h, w = im.shape[:2]
    return w, h

def resize_longside(im: Image.Image, longside=960):
    w, h = im.size
    s = longside / max(w, h)
    if s >= 1.0:
        return im
    return im.resize((int(round(w*s)), int(round(h*s))), Image.BICUBIC)


def local_matching(imgA_path, imgB_path, longside=960, max_kps=4096, thr=0.2):

    A = resize_longside(Image.open(imgA_path).convert("RGB"), longside=longside)
    B = resize_longside(Image.open(imgB_path).convert("RGB"), longside=longside)
    images = [A, B]
    image_sizes = [[(im.height, im.width) for im in images]]

    inputs = local_processor(images, return_tensors="pt")
    inputs = {k: (v.to(device) if hasattr(v, "to") else v) for k, v in inputs.items()}

    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type == "cuda")):
        outputs = local_model(**inputs)

    matches_list = local_processor.post_process_keypoint_matching(outputs, image_sizes, threshold=thr)
    if not matches_list:
        return (np.empty((0,2), np.float32),
                np.empty((0,2), np.float32),
                np.empty((0,), np.float32),
                (A.size, B.size))

    m = matches_list[0]

    def to_numpy(t):
        if hasattr(t, "detach"):
            return t.detach().cpu().numpy().astype(np.float32)
        return np.asarray(t, dtype=np.float32)

    kpts0  = to_numpy(m["keypoints0"])
    kpts1  = to_numpy(m["keypoints1"])
    scores = to_numpy(m.get("matching_scores", np.zeros((len(kpts0),), dtype=np.float32)))

    if max_kps is not None and len(scores) > max_kps:
        idx = np.argsort(scores)[-max_kps:]
        kpts0, kpts1, scores = kpts0[idx], kpts1[idx], scores[idx]

    return kpts0, kpts1, scores, (A.size, B.size)


folder = "train/ETs"
paths, X, pairs = global_image_pairing(
        folder,
        batch_size=60,
        K=40,
        use_pca=True,
        pca_dim=256,
        jaccard_min=0.25,
    )







[PCA] Reducing out_dim from 256 -> 21 (N=22, D=1536)
[('test/ETs/et_et005.png', 'test/ETs/outliers_out_et003.png'), ('test/ETs/another_et_another_et007.png', 'test/ETs/et_et008.png'), ('test/ETs/et_et006.png', 'test/ETs/outliers_out_et002.png'), ('test/ETs/another_et_another_et008.png', 'test/ETs/et_et007.png'), ('test/ETs/another_et_another_et005.png', 'test/ETs/another_et_another_et010.png'), ('test/ETs/another_et_another_et004.png', 'test/ETs/et_et003.png'), ('test/ETs/another_et_another_et006.png', 'test/ETs/et_et000.png'), ('test/ETs/another_et_another_et009.png', 'test/ETs/another_et_another_et010.png'), ('test/ETs/another_et_another_et001.png', 'test/ETs/another_et_another_et006.png'), ('test/ETs/another_et_another_et006.png', 'test/ETs/outliers_out_et001.png'), ('test/ETs/another_et_another_et009.png', 'test/ETs/et_et008.png'), ('test/ETs/et_et007.png', 'test/ETs/outliers_out_et003.png'), ('test/ETs/another_et_another_et010.png', 'test/ETs/et_et007.png'), ('test/ETs/another_et_

In [3]:
# 2) Write features + tentative matches into DB
from sklearn.neighbors import NearestNeighbors
db = pycolmap.Database("colmap.db")
db.clear_all_tables()   # drops all tables’ contents
db.close()
"""
# -------------------------------
# 1) Simple COLMAP DB helpers
# -------------------------------
class COLMAPDatabase(pycolmap.Database):  # thin convenience wrapper
    pass

def open_db(db_path):
    if os.path.exists(db_path):
        os.remove(db_path)
    db = pycolmap.Database(); 
    db.open(db_path)
    return db

def add_image_and_camera(db: pycolmap.Database, image_path: str, default_f: float = 1200.0) -> tuple[int, tuple[int,int]]:
    from PIL import Image
    im = Image.open(image_path)
    w, h = im.size
    fx = fy = float(default_f)
    cx, cy = w / 2.0, h / 2.0

    cam = pycolmap.Camera(
        model='SIMPLE_PINHOLE',
        width=w, height=h,
        params=np.array([fx, cx, cy], dtype=np.float64)
    )
    cam_id = db.write_camera(cam)

    img = pycolmap.Image(name=Path(image_path).name, camera_id=cam_id)
    image_id = db.write_image(img)
    return image_id, (w, h)

# -----------------------------------------
# 2) Global per-image keypoint store/merger
# -----------------------------------------
class KeypointStore:

    def __init__(self, merge_px=2.0):
        self.kpts = {}         # image_name -> np.ndarray [N,2]
        self.kdtrees = {}      # optional speed-ups (lazy)
        self.merge_px2 = merge_px**2

    def _ensure_image(self, image_name):
        if image_name not in self.kpts:
            self.kpts[image_name] = np.zeros((0, 2), dtype=np.float32)
            self.kdtrees[image_name] = None

    def _nn_match(self, image_name, pts):

        base = self.kpts[image_name]
        if len(base) == 0 or len(pts) == 0:
            return np.full(len(pts), -1, np.int64), np.full(len(pts), np.inf, np.float32)
        # (M,2) vs (N,2)
        d2 = ((pts[:,None,:] - base[None,:,:])**2).sum(-1)  # (M,N)
        best_idx = d2.argmin(axis=1)
        best_d2 = d2[np.arange(len(pts)), best_idx]
        return best_idx.astype(np.int64), best_d2.astype(np.float32)

    def assign_indices(self, image_name, pts):
 
        self._ensure_image(image_name)
        pts = pts.astype(np.float32)

        # find nearest existing kpt
        nn_idx, nn_d2 = self._nn_match(image_name, pts)

        # Decide which are new (distance too large)
        is_new = nn_d2 > self.merge_px2
        idx = nn_idx.copy()
        if np.any(is_new):
            base = self.kpts[image_name]
            start = len(base)
            new_pts = pts[is_new]
            self.kpts[image_name] = np.vstack([base, new_pts])
            # indices for new ones
            idx[is_new] = np.arange(start, start + len(new_pts), dtype=np.int64)

        return idx

# -----------------------------------------------------
# 3) Build DB: add images, keypoints, and tentative matches
# -----------------------------------------------------
def write_db_from_local_matches(
    db_path,
    image_paths,
    pairs,
    match_func,
    min_matches_keep=15,
    merge_px=2.0,
    verbose=True
):


    db = open_db(db_path)

    # map name->id and keep sizes
    name2id = {}
    name2size = {}
    for p in image_paths:
        img_id, (w, h) = add_image_and_camera(db, p)
        name2id[Path(p).name] = img_id
        name2size[Path(p).name] = (w, h)

    store = KeypointStore(merge_px=merge_px)

    # We will collect per-image keypoints through the loop,
    # but we can only call db.add_keypoints ONCE per image (after we finalize them).
    # So we buffer matches first as "index pairs" and add keypoints at the end.
    buffered_matches = defaultdict(list)  # (nameA,nameB) -> list of (i_idx, j_idx) arrays to concat

    for a, b in pairs:
        nameA = Path(a).name
        nameB = Path(b).name
        try:
            k0, k1, scores, _ = match_func(a, b)
            if len(k0) == 0:
                continue

            # Assign stable indices in the per-image stores
            idxA = store.assign_indices(nameA, k0)  # (M,)
            idxB = store.assign_indices(nameB, k1)  # (M,)

            # (optional) keep only top matches by score here if you want
            # e.g., top_idx = np.argsort(scores)[-4096:]
            # idxA, idxB = idxA[top_idx], idxB[top_idx]

            match_ij = np.stack([idxA, idxB], axis=1).astype(np.uint32)
            if len(match_ij) >= min_matches_keep:
                buffered_matches[(nameA, nameB)].append(match_ij)

            if verbose:
                print(f"[pair ok] {nameA} - {nameB} matches={len(match_ij)}")

        except Exception as e:
            print(f"[pair fail] {nameA} - {nameB}: {e}")

    # Now insert keypoints (one shot per image)
    for name, kpts in store.kpts.items():
        img_id = name2id[name]
        db.add_keypoints(img_id, kpts.astype(np.float32))

    # Insert matches
    for (nameA, nameB), chunks in buffered_matches.items():
        if not chunks:
            continue
        ij = np.vstack(chunks).astype(np.uint32)

        idA, idB = name2id[nameA], name2id[nameB]
        # enforce id order for COLMAP
        if idA > idB:
            id1, id2 = idB, idA
            matches = ij[:, [1, 0]]
        else:
            id1, id2 = idA, idB
            matches = ij

        db.add_matches(id1, id2, matches)

    db.commit()
    db.close()


    # 1) Build your prefiltered_pairs (list of (A,B))
def keep_pair(k0, k1, scores,
              min_matches=25, mean_thr=0.25, std_max=0.35):
    if len(scores) < min_matches:
        return False
    m = float(scores.mean()) if len(scores) else 0.0
    s = float(scores.std())  if len(scores) else 1.0
    return (m >= mean_thr) and (s <= std_max)

"""



# --- Utilities ---

def get_original_wh(img_path):
    w, h = Image.open(img_path).size
    return float(w), float(h)

def scale_keypoints_to_original(kpts_pair_resized, orig_w, orig_h, res_w, res_h):
    # Keypoints are (x,y) in resized image; map back to original pixel coords
    sx, sy = orig_w / res_w, orig_h / res_h
    return (kpts_pair_resized * np.array([sx, sy], np.float32)).astype(np.float32)

class GlobalKeypointIndexer:
    """
    Maintains a per-image global keypoint array and provides
    mapping from new (pair-local) keypoints to global indices,
    merging points that are within 'merge_px' in the *original resolution*.
    """
    def __init__(self, merge_px=2.0):
        self.kpts = {}          # img_path -> np.ndarray [N,2] float32 (original-res coords)
        self.nn = {}            # img_path -> NearestNeighbors
        self.merge_px = float(merge_px)

    def _fit_nn(self, img):
        if len(self.kpts[img]) == 0:
            self.nn[img] = None
            return
        nn = NearestNeighbors(n_neighbors=1, algorithm='auto')
        nn.fit(self.kpts[img])
        self.nn[img] = nn

    def _ensure(self, img):
        if img not in self.kpts:
            self.kpts[img] = np.empty((0, 2), np.float32)
            self.nn[img] = None

    def map_and_add(self, img, new_pts):
        """
        new_pts: (M,2) float32 in original-res coords for 'img'
        Returns: (M,) int64 global indices for each input point
        """
        self._ensure(img)
        if new_pts.size == 0:
            return np.empty((0,), np.int64)

        if self.nn[img] is None or len(self.kpts[img]) == 0:
            # first points for this image
            start = len(self.kpts[img])
            self.kpts[img] = np.vstack([self.kpts[img], new_pts])
            idx = np.arange(start, start + len(new_pts), dtype=np.int64)
            self._fit_nn(img)
            return idx

        # query nearest existing to decide merge vs append
        dists, nbrs = self.nn[img].kneighbors(new_pts, n_neighbors=1, return_distance=True)
        dists = dists.ravel()
        nbrs = nbrs.ravel()

        to_merge = dists <= self.merge_px
        to_add   = ~to_merge

        idx = np.empty((len(new_pts),), np.int64)
        # merged points keep neighbor's index
        idx[to_merge] = nbrs[to_merge]

        new_pts_to_add = new_pts[to_add]
        start = len(self.kpts[img])
        self.kpts[img] = np.vstack([self.kpts[img], new_pts_to_add])
        idx[to_add] = np.arange(start, start + len(new_pts_to_add), dtype=np.int64)

        self._fit_nn(img)
        return idx

# --- Glue: accumulate global keypoints and write DB ---

def build_db_from_local_matches(db_path, pairs, local_match_func,
                                longside=960, max_kps=4096, thr=0.2,
                                merge_px=2.0, default_fov_px=1_200.0, verbose=True):
    """
    pairs: list of (imgA_path, imgB_path)
    local_match_func(imgA, imgB, longside, max_kps, thr) -> kpts0_resized, kpts1_resized, scores, ((resW_A,resH_A),(resW_B,resH_B))
    Writes:
      - global per-image keypoints (original resolution)
      - matches as indices into those global arrays
    """
    db = pycolmap.Database(db_path)

    img_name_to_id = {}
    img_name_to_wh = {}
    img_name_to_cam = {}

    #get all unique image names
    all_images = sorted({p for ab in pairs for p in ab})
    for img_path in all_images:
        w, h = get_original_wh(img_path)
        img_name = Path(img_path).name
        # camera: fx = fy = default_fov_px, cx = w/2, cy = h/2
        cam = pycolmap.Camera(model='SIMPLE_PINHOLE',
                              width=int(w), height=int(h),
                              params=np.array([float(default_fov_px), w/2.0, h/2.0], dtype=np.float64))
        cam_id = db.write_camera(cam)
        img_row = pycolmap.Image(name=img_name, camera_id=cam_id)
        img_id = db.write_image(img_row)

        img_name_to_id[img_name] = img_id
        img_name_to_wh[img_name] = (w, h)
        img_name_to_cam[img_name] = cam_id
    indexer = GlobalKeypointIndexer(merge_px=merge_px)

    # Store pair matches as global indices to write later
    pending_matches = []

    for a_path, b_path in pairs:
        k0_r, k1_r, scores, ((resW_A, resH_A), (resW_B, resH_B)) = local_match_func(
            a_path, b_path, longside=longside, max_kps=max_kps, thr=thr
        )

        nameA = Path(a_path).name
        nameB = Path(b_path).name
        wA, hA = img_name_to_wh[nameA]
        wB, hB = img_name_to_wh[nameB]
        k0 = scale_keypoints_to_original(k0_r, wA, hA, float(resW_A), float(resH_A))
        k1 = scale_keypoints_to_original(k1_r, wB, hB, float(resW_B), float(resH_B))

        # map to global indices with deduplication
        idxA = indexer.map_and_add(a_path, k0)
        idxB = indexer.map_and_add(b_path, k1)

        if len(idxA) > 0:
            ij = np.stack([idxA.astype(np.uint32), idxB.astype(np.uint32)], axis=1)
            pending_matches.append((img_name_to_id[nameA], img_name_to_id[nameB], ij))

        if verbose:
            print(f"[pair] {nameA} ↔ {nameB}: {len(idxA)} tentative (global) matches")

    for img_path, kpts in indexer.kpts.items():
        name = Path(img_path).name
        img_id = img_name_to_id[name]
        db.write_keypoints(img_id, kpts.astype(np.float32))
        if verbose:
            print(f"[kpts] {name}: {len(kpts)} keypoints")

    for idA, idB, ij in pending_matches:
        db.write_matches(idA, idB, ij.astype(np.uint32))

    db.close()

    if verbose:
        print(f"[done] wrote database to {db_path}")




db_path = "colmap.db"
build_db_from_local_matches(db_path, pairs, local_matching,
                                longside=960, max_kps=4096, thr=0.2,
                                merge_px=2.0, default_fov_px=1_200.0, verbose=True)

    
"""
prefiltered_pairs = []
all_images = sorted({p for pair in pairs for p in pair})

for a, b in pairs:
    k0, k1, sc, _ = local_matching(a, b, longside=960, max_kps=4096, thr=0.2)
    if keep_pair(k0, k1, sc):
        prefiltered_pairs.append((a, b))

write_db_from_local_matches(
    db_path=db_path,
    image_paths=all_images,
    pairs=prefiltered_pairs,
    match_func=lambda A,B: local_matching(A,B, longside=960, max_kps=4096, thr=0.2),
    min_matches_keep=20,   # loose; pycolmap.verify_matches will be stricter
    merge_px=2.0,
    verbose=True
)

# 3) Let COLMAP do the official geometric verification (fast, C++)
from pathlib import Path

pairs_txt = "pairs.txt"
with open(pairs_txt, "w") as f:
    for a,b in prefiltered_pairs:
        f.write(f"{Path(a).name} {Path(b).name}\n")
"""


[pair] et_et005.png ↔ outliers_out_et003.png: 10 tentative (global) matches
[pair] another_et_another_et007.png ↔ et_et008.png: 16 tentative (global) matches
[pair] et_et006.png ↔ outliers_out_et002.png: 4 tentative (global) matches
[pair] another_et_another_et008.png ↔ et_et007.png: 9 tentative (global) matches
[pair] another_et_another_et005.png ↔ another_et_another_et010.png: 16 tentative (global) matches
[pair] another_et_another_et004.png ↔ et_et003.png: 11 tentative (global) matches
[pair] another_et_another_et006.png ↔ et_et000.png: 2 tentative (global) matches
[pair] another_et_another_et009.png ↔ another_et_another_et010.png: 62 tentative (global) matches
[pair] another_et_another_et001.png ↔ another_et_another_et006.png: 152 tentative (global) matches
[pair] another_et_another_et006.png ↔ outliers_out_et001.png: 10 tentative (global) matches
[pair] another_et_another_et009.png ↔ et_et008.png: 13 tentative (global) matches
[pair] et_et007.png ↔ outliers_out_et003.png: 6 tentat

'\nprefiltered_pairs = []\nall_images = sorted({p for pair in pairs for p in pair})\n\nfor a, b in pairs:\n    k0, k1, sc, _ = local_matching(a, b, longside=960, max_kps=4096, thr=0.2)\n    if keep_pair(k0, k1, sc):\n        prefiltered_pairs.append((a, b))\n\nwrite_db_from_local_matches(\n    db_path=db_path,\n    image_paths=all_images,\n    pairs=prefiltered_pairs,\n    match_func=lambda A,B: local_matching(A,B, longside=960, max_kps=4096, thr=0.2),\n    min_matches_keep=20,   # loose; pycolmap.verify_matches will be stricter\n    merge_px=2.0,\n    verbose=True\n)\n\n# 3) Let COLMAP do the official geometric verification (fast, C++)\nfrom pathlib import Path\n\npairs_txt = "pairs.txt"\nwith open(pairs_txt, "w") as f:\n    for a,b in prefiltered_pairs:\n        f.write(f"{Path(a).name} {Path(b).name}\n")\n'

In [12]:
def run_pycolmap_verify_and_map(db_path: str, image_root: str, out_dir: str):
    """
    1) Geometric verification in C++ (populates two-view geometry).
    2) Incremental mapping to produce reconstructions.
    """
    db = pycolmap.Database(db_path)
    pair_ids = db.read_two_view_geometry_num_inliers()[0]  #verified
    if len(pair_ids) == 0:
        pair_ids = db.read_all_matches()[0]
    db.close()

    db = pycolmap.Database(db_path)
    id2name = {img.image_id: img.name for img in db.read_all_images()}
    pairs_txt = Path(out_dir) / "pairs.txt"
    pairs_txt.parent.mkdir(parents=True, exist_ok=True)
    with pairs_txt.open("w") as f:
        for pid in pair_ids:
            i, j = pycolmap.Database.pair_id_to_image_pair(pid)
            f.write(f"{id2name[i]} {id2name[j]}\n")
    db.close()

    # 1) geometric verification
    pycolmap.verify_matches(
        database_path=db_path,
        pairs_path=str(pairs_txt),
        options=pycolmap.TwoViewGeometryOptions()  # tune if needed
    )

    # 2) incremental mapping
    mapper_opts = pycolmap.IncrementalPipelineOptions()
    mapper_opts.min_model_size = 3
    mapper_opts.min_num_matches = 30
    recon_dir = Path(out_dir) / "sparse"
    recon_dir.mkdir(parents=True, exist_ok=True)

    maps = pycolmap.incremental_mapping(
        database_path=db_path,
        image_path=image_root,
        output_path=str(recon_dir),
        options=mapper_opts
    )
    return maps
maps = run_pycolmap_verify_and_map(
    db_path=db_path,
    image_root=folder,
    out_dir="colmap_outputs"
)

print("Reconstructions:", len(maps))

I20251014 23:23:02.010472 126058120586816 misc.cc:44] 
Feature matching
I20251014 23:23:02.014770 126058128979520 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.015120 126058533746240 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.015423 126058380645952 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.015930 126058657478208 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.016322 126058649085504 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.016709 126058137372224 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.017004 126058525353536 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.017344 126058516960832 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.017708 126058397431360 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.018072 126058389038656 sift.cc:1434] Creating SIFT CPU feature matcher
I20251014 23:23:02.018447 12

Reconstructions: 2


I20251014 23:23:03.657375 126066262986752 incremental_pipeline.cc:442] => Could not register, trying another image.
I20251014 23:23:03.657412 126066262986752 incremental_pipeline.cc:428] Registering image #17 (num_reg_frames=10)
I20251014 23:23:03.657423 126066262986752 incremental_pipeline.cc:431] => Image sees 53 / 429 points
I20251014 23:23:03.661606 126066262986752 incremental_pipeline.cc:442] => Could not register, trying another image.
I20251014 23:23:03.661650 126066262986752 incremental_pipeline.cc:43] Retriangulation and Global bundle adjustment
I20251014 23:23:03.666985 126066262986752 incremental_pipeline.cc:584] Keeping successful reconstruction
I20251014 23:23:03.667090 126066262986752 incremental_pipeline.cc:235] => Relaxing the initialization constraints.
I20251014 23:23:03.667209 126066262986752 incremental_pipeline.cc:300] Finding good initial image pair
I20251014 23:23:03.667217 126066262986752 incremental_pipeline.cc:304] => No good initial image pair found.
I2025101

In [32]:
import csv
import math
import pandas as pd

### UTILISING PYCOLMAP DOCUMENTATION -> https://colmap.github.io/format.html
### EVALUATION IS ACCORDING TO IMC CHALLENGE 2024!
"""def camera_centers(Rs, ts):
    Rt = np.transpose(Rs, (0, 2, 1))
    return (-Rt @ ts[..., None])[..., 0]

def umeyama_similarity(X, Y, with_scale=True): ## Horn's method
    X = np.asarray(X, np.float64)
    Y = np.asarray(Y, np.float64)
    muX, muY = X.mean(0), Y.mean(0)
    Xc, Yc = X - muX, Y - muY
    C = (Yc.T @ Xc) / X.shape[0]
    U, S, Vt = np.linalg.svd(C)
    R = U @ np.diag([1, 1, np.sign(np.linalg.det(U @ Vt))]) @ Vt
    s = 1.0
    if with_scale:
        var_X = (Xc**2).sum() / X.shape[0]
        s = (S * np.array([1, 1, np.sign(np.linalg.det(U @ Vt))])).sum() / var_X
    t = muY - s * (R @ muX)
    return float(s), R, t

def apply_similarity(C, s, R, t):
    return (s * (C @ R.T)) + t

def best_center_alignment(C_pred, C_gt, sample_mode="ransac", n_iters=2000, rng=0):
    N = len(C_pred)
    rng = np.random.default_rng(rng)
    # 2% of scene extent as selection threshold for model choice
    sel_thresh = 0.02 * np.linalg.norm(C_gt.max(0) - C_gt.min(0))
    best_score, best = -1, (1.0, np.eye(3), np.zeros(3), np.zeros(N, bool))

    # If scene is small, try exhaustive triplets; else random RANSAC
    max_trip = 150_000
    use_exhaustive = math.comb(N, 3) <= max_trip
    if sample_mode == "exhaustive" and use_exhaustive:
        from itertools import combinations
        triplets = combinations(range(N), 3)
    else:
        triplets = [tuple(rng.choice(N, 3, replace=False)) for _ in range(n_iters)]

    for tri in triplets:
        i, j, k = tri
        s, R, t = umeyama_similarity(C_pred[[i, j, k]], C_gt[[i, j, k]], with_scale=True)
        C_al = apply_similarity(C_pred, s, R, t)
        inl = np.linalg.norm(C_al - C_gt, axis=1) <= sel_thresh
        sc = int(inl.sum())
        if sc > best_score:
            best_score = sc
            best = (s, R, t, inl)
    return best"""

"""def compute_mAA(R_pred, t_pred, R_gt, t_gt, thresholds_m=None, seed=0):
    R_pred = np.asarray(R_pred, float); t_pred = np.asarray(t_pred, float)
    R_gt   = np.asarray(R_gt, float);   t_gt   = np.asarray(t_gt, float)
    assert R_pred.shape == R_gt.shape and t_pred.shape == t_gt.shape
    C_pred = camera_centers(R_pred, t_pred)
    C_gt   = camera_centers(R_gt,   t_gt)

    s, R, t, _ = best_center_alignment(C_pred, C_gt, sample_mode="ransac", n_iters=2000, rng=seed)
    C_al = apply_similarity(C_pred, s, R, t)
    d = np.linalg.norm(C_al - C_gt, axis=1)

    if thresholds_m is None:
        thresholds_m = [0.01, 0.02, 0.05, 0.1, 0.2]  # meters (example)
    accs = [(d <= thr).mean() for thr in thresholds_m]
    return {
        "mAA": float(np.mean(accs)),
        "per_threshold": dict(zip(thresholds_m, accs)),
        "similarity": {"scale": s, "R": R, "t": t},
        "aligned_centers": C_al,
        "center_errors": d,
    }"""

_EPS = np.finfo(float).eps * 4.0

def vector_norm(data, axis=None, out=None):
    data = np.array(data, dtype=np.float64, copy=True)
    if out is None:
        if data.ndim == 1:
            return math.sqrt(np.dot(data, data))
        data *= data
        out = np.atleast_1d(np.sum(data, axis=axis))
        np.sqrt(out, out)
        return out
    data *= data
    np.sum(data, axis=axis, out=out)
    np.sqrt(out, out)
    return None



def quaternion_matrix(quaternion):
    q = np.array(quaternion, dtype=np.float64, copy=True)
    n = np.dot(q, q)
    if n < _EPS:
        return np.identity(4)
    q *= math.sqrt(2.0 / n)
    q = np.outer(q, q)
    return np.array(
        [
            [1.0 - q[2, 2] - q[3, 3], q[1, 2] - q[3, 0],     q[1, 3] + q[2, 0],     0.0],
            [q[1, 2] + q[3, 0],     1.0 - q[1, 1] - q[3, 3], q[2, 3] - q[1, 0],     0.0],
            [q[1, 3] - q[2, 0],     q[2, 3] + q[1, 0],     1.0 - q[1, 1] - q[2, 2], 0.0],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )



def affine_matrix_from_points(v0, v1, shear=False, scale=True, usesvd=True):
    v0 = np.array(v0, dtype=np.float64, copy=True)
    v1 = np.array(v1, dtype=np.float64, copy=True)
    ndims = v0.shape[0]
    if ndims < 2 or v0.shape[1] < ndims or v0.shape != v1.shape:
        raise ValueError("input arrays are of wrong shape or type")
    # center
    t0 = -np.mean(v0, axis=1); M0 = np.identity(ndims + 1); M0[:ndims, ndims] = t0; v0 += t0.reshape(ndims, 1)
    t1 = -np.mean(v1, axis=1); M1 = np.identity(ndims + 1); M1[:ndims, ndims] = t1; v1 += t1.reshape(ndims, 1)
    if shear:
        A = np.concatenate((v0, v1), axis=0)
        u, s, vh = np.linalg.svd(A.T)
        vh = vh[:ndims].T
        B = vh[:ndims]; C = vh[ndims: 2 * ndims]
        t = np.dot(C, np.linalg.pinv(B))
        t = np.concatenate((t, np.zeros((ndims, 1))), axis=1)
        M = np.vstack((t, ((0.0,) * ndims) + (1.0,)))
    elif usesvd or ndims != 3:
        u, s, vh = np.linalg.svd(np.dot(v1, v0.T))
        R = np.dot(u, vh)
        if np.linalg.det(R) < 0.0:
            R -= np.outer(u[:, ndims - 1], vh[ndims - 1, :] * 2.0)
            s[-1] *= -1.0
        M = np.identity(ndims + 1)
        M[:ndims, :ndims] = R
    else:
        xx, yy, zz = np.sum(v0 * v1, axis=1)
        xy, yz, zx = np.sum(v0 * np.roll(v1, -1, axis=0), axis=1)
        xz, yx, zy = np.sum(v0 * np.roll(v1, -2, axis=0), axis=1)
        N = [
            [xx + yy + zz, 0.0, 0.0, 0.0],
            [yz - zy, xx - yy - zz, 0.0, 0.0],
            [zx - xz, xy + yx, yy - xx - zz, 0.0],
            [xy - yx, zx + xz, yz + zy, zz - xx - yy],
        ]
        w, V = np.linalg.eigh(N); q = V[:, np.argmax(w)]; q /= vector_norm(q)
        M = quaternion_matrix(q)
    if scale and not shear:
        v0 *= v0; v1 *= v1
        M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))
    M = np.dot(np.linalg.inv(M1), np.dot(M, M0))
    M /= M[ndims, ndims]
    return M

def register_by_Horn(ev_coord, gt_coord, ransac_threshold, inl_cf, strict_cf):
    # Filter invalid
    idx_cams = np.all(np.isfinite(ev_coord), axis=0)
    ev_coord = ev_coord[:, idx_cams]
    gt_coord = gt_coord[:, idx_cams]

    n = ev_coord.shape[1]
    r = ransac_threshold.shape[0]
    ransac_threshold = np.expand_dims(ransac_threshold, axis=0)
    ransac_threshold2 = ransac_threshold**2
    ev_coord_1 = np.vstack((ev_coord, np.ones(n)))

    max_no_inl = np.zeros((1, r))
    best_inl_err = np.full(r, np.Inf)
    best_transf_matrix = np.zeros((r, 4, 4))
    best_err = np.full((n, r), np.Inf)
    strict_inl = np.full((n, r), False)
    triplets_used = np.zeros((3, r))

    for ii in range(n-2):
        for jj in range(ii+1, n-1):
            for kk in range(jj+1, n):
                i = [ii, jj, kk]
                if np.all(strict_inl[i]):
                    continue
                T = affine_matrix_from_points(ev_coord[:, i], gt_coord[:, i], usesvd=False)
                rotranslated = np.matmul(T[:3], ev_coord_1)
                err = np.sum((rotranslated - gt_coord)**2, axis=0)  # squared
                inl = np.expand_dims(err, axis=1) < ransac_threshold2
                no_inl = np.sum(inl, axis=0)
                to_ref = np.squeeze(((no_inl > 2) & (no_inl > max_no_inl * inl_cf)), axis=0)
                for q in np.argwhere(to_ref):
                    qq = q[0]
                    if np.any(np.all((np.expand_dims(inl[:, qq], axis=1) == inl[:, :qq]), axis=0)):
                        continue
                    T = affine_matrix_from_points(ev_coord[:, inl[:, qq]], gt_coord[:, inl[:, qq]])
                    rotranslated = np.matmul(T[:3], ev_coord_1)
                    err_ref = np.sum((rotranslated - gt_coord)**2, axis=0)
                    err_ref_sum = np.sum(err_ref, axis=0)
                    err_ref = np.expand_dims(err_ref, axis=1)
                    inl_ref = err_ref < ransac_threshold2
                    no_inl_ref = np.sum(inl_ref, axis=0)
                    to_update = np.squeeze((no_inl_ref > max_no_inl) | ((no_inl_ref == max_no_inl) & (err_ref_sum < best_inl_err)), axis=0)
                    if np.any(to_update):
                        triplets_used[0, to_update] = ii
                        triplets_used[1, to_update] = jj
                        triplets_used[2, to_update] = kk
                        max_no_inl[:, to_update] = no_inl_ref[to_update]
                        best_err[:, to_update] = np.sqrt(err_ref)  # back to euclidean
                        best_inl_err[to_update] = err_ref_sum
                        strict_inl[:, to_update] = (best_err[:, to_update] < (strict_cf * ransac_threshold[:, to_update]))
                        best_transf_matrix[to_update] = T

    best_model = {
        "valid_cams": idx_cams,
        "no_inl": max_no_inl,
        "err": best_err,
        "triplets_used": triplets_used,
        "transf_matrix": best_transf_matrix
    }
    return best_model




def mAA_on_cameras(err, thresholds, n, skip_top_thresholds, to_dec):
    aux = err[:, skip_top_thresholds:] < np.expand_dims(np.asarray(thresholds[skip_top_thresholds:]), axis=0)
    return np.sum(np.maximum(np.sum(aux, axis=0) - to_dec, 0)) / (len(thresholds[skip_top_thresholds:]) * (n - to_dec))
    
def get_camera_centers_from_df(df):
    out = {}
    for _, row in df.iterrows():
        fname = row['image_path']
        R = np.array([float(x) for x in row['rotation_matrix'].split(';')]).reshape(3,3)
        t = np.array([float(x) for x in row['translation_vector'].split(';')]).reshape(3)
        center = -R.T @ t
        out[fname] = center
    return out

def evaluate_rec(gt_df, user_df, thresholds, inl_cf=0.8, strict_cf=0.5, skip_top_thresholds=2, to_dec=3, verbose=True):
    ucameras = get_camera_centers_from_df(user_df)
    gcameras = get_camera_centers_from_df(gt_df)

    good = [k for k in gcameras.keys() if k in ucameras]
    n = len(good)
    u = np.zeros((3, n)); g = np.zeros((3, n))
    for i, k in enumerate(good):
        u[:, i] = ucameras[k]
        g[:, i] = gcameras[k]

    model = register_by_Horn(u, g, np.asarray(thresholds), inl_cf, strict_cf)

    if verbose and len(thresholds) > 0:
        T = np.squeeze(model['transf_matrix'][-1])
        print("\n[eval] Similarity (max threshold) transform:\n", T)

    m = gt_df.shape[0]
    mAA = mAA_on_cameras(model["err"], thresholds, m, skip_top_thresholds, to_dec)
    return mAA, model


# --- Quaternion -> rotation (right-handed, COLMAP qvec = [qw, qx, qy, qz]) ---
def qvec_to_rotmat(q):
    qw, qx, qy, qz = map(float, q)
    # normalized just in case
    s = qw*qw + qx*qx + qy*qy + qz*qz
    if s == 0:
        return np.eye(3, dtype=np.float64)
    qw, qx, qy, qz = qw/np.sqrt(s), qx/np.sqrt(s), qy/np.sqrt(s), qz/np.sqrt(s)
    R = np.array([
        [1-2*(qy*qy+qz*qz),   2*(qx*qy - qz*qw),   2*(qx*qz + qy*qw)],
        [2*(qx*qy + qz*qw),   1-2*(qx*qx+qz*qz),   2*(qy*qz - qx*qw)],
        [2*(qx*qz - qy*qw),   2*(qy*qz + qx*qw),   1-2*(qx*qx+qy*qy)]
    ], dtype=np.float64)
    return R

def read_reconstruction_poses(model_dir,dataset,scene):
    """
    Returns: dict[name] = {'R': (3,3), 't': (3,), 'C': (3,)} 
    where [R|t] maps world -> camera, and C is camera center in world coord.
    """
    rows = []
    images_txt_path = model_dir + "/" + "images.txt"
    with open(images_txt_path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip() and not ln.startswith("#")]
    for i in range(0, len(lines), 2):
        hdr = lines[i].split()

        # IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
        image_id = int(hdr[0])
        qw, qx, qy, qz = map(float, hdr[1:5])
        tx, ty, tz = map(float, hdr[5:8])
        cam_id = int(hdr[8])
        name = " ".join(hdr[9:])
        q = np.array([qw, qx, qy, qz], dtype=np.float64)
        t = np.array([tx, ty, tz], dtype=np.float64)

        R = qvec_to_rotmat(q)

        r_str = ";".join(f"{x:.9f}" for x in R.reshape(-1).tolist())
        t_str = ";".join(f"{x:.9f}" for x in t.reshape(-1).tolist())
        rows.append({
            "dataset": dataset,
            "scene": scene,
            "image_path": name,
            "rotation_matrix": r_str,
            "translation_vector": t_str
        })
    return pd.DataFrame(rows)

def parse_semicolon_vec(s):
    return np.array([float(x) for x in s.split(";")], dtype=float)

def parse_semicolon_mat9(s):
    # 9 values row-major separated by ';'
    vals = [float(x) for x in s.split(";")]
    assert len(vals) == 9, f"Expected 9 values, got {len(vals)}"
    return np.array(vals, dtype=float).reshape(3, 3)

def read_gt_csv(path, dataset=None, scene=None):
    """
    Returns dict: {basename: (R_gt, t_gt)}.
    Expects columns: dataset,scene,image,rotation_matrix,translation_vector
    """
    out = {}
    with open(path, "r", newline="") as f:
        rdr = csv.DictReader(f)
        for row in rdr:
            if dataset and row.get("dataset") != dataset:
                continue
            if scene and row.get("scene") != scene:
                continue
            name = os.path.basename(row["image"])
            R = parse_semicolon_mat9(row["rotation_matrix"])
            t = parse_semicolon_vec(row["translation_vector"])
            out[name] = (R, t)
    return out


def evaluate_mAA_from_model(model_dir, gt_csv_path, thresholds_csv,
                            dataset=None, scene=None, seed=0, verbose=True):
    """
    model_dir: path to COLMAP reconstruction (e.g., 'path/to/sparse/0')
    gt_csv_path: CSV with GT poses
    thresholds_m: iterable of thresholds in meters (or your linear unit)
    dataset/scene: optional filters for GT rows
    """

    inl_cf=0.8
    strict_cf=0.5
    skip_top_thresholds=0
    to_dec=3
    pred_df = read_reconstruction_poses(model_dir,dataset,scene)
    gt = []
    with open(gt_csv_path, "r", newline="") as f:
        for row in csv.DictReader(f):
            if row["dataset"].strip() != dataset or row["scene"].strip() != scene:
                continue
            gt.append({
                "dataset": row["dataset"].strip(),
                "scene": row["scene"].strip(),
                "image_path": row["image"].strip() if "image" in row else row["image_path"].strip(),
                "rotation_matrix": row["rotation_matrix"].strip(),
                "translation_vector": row["translation_vector"].strip(),
            })
    gt_df = pd.DataFrame(gt)


    thr_map = read_thresholds_csv(thresholds_csv) if thresholds_csv else None
    thresholds = thresholds_for(thr_map, dataset, scene)

    mAA, model = evaluate_rec(
        gt_df, pred_df,
        thresholds=thresholds,
        inl_cf=inl_cf,
        strict_cf=strict_cf,
        skip_top_thresholds=skip_top_thresholds,
        to_dec=to_dec,
        verbose=verbose
    )
    if verbose:
        print(f"\n[mAA] dataset={dataset} scene={scene} => {mAA*100:.2f}%")

    return mAA, model, pred_df, gt_df

def read_thresholds_csv(path):
    """
    Returns dict[(dataset, scene)] = list_of_thresholds(floats)
    CSV columns: dataset,scene,thresholds (semicolon-separated meters)
    """
    out = {}
    with open(path, "r", newline="") as f:
        for row in csv.DictReader(f):
            ds = row["dataset"].strip()
            sc = row["scene"].strip()
            thr = [float(x) for x in row["thresholds"].split(";") if x.strip()]
            out[(ds, sc)] = thr
    return out
def thresholds_for(thr_map, dataset, scene, fallback=(0.01, 0.02, 0.05, 0.1, 0.2)):
    return thr_map.get((dataset, scene), list(fallback))



model_dir = "colmap_outputs/sparse/0"
model_dir1 = "colmap_outputs/sparse/1"
gt_csv = "train_labels.csv"
threshold_csv = "train_thresholds.csv"

thresholds_map = read_thresholds_csv(threshold_csv)
ds, sc = "ETs", "ET"
ds1, sc1 = "ETs", "another_ET"





mAA, model, pred_df, gt_df = evaluate_mAA_from_model(
model_dir, gt_csv,
threshold_csv,
dataset=ds, scene=sc, verbose=True
 )
print("mAA:",mAA)




mAA1, model1, pred_df1, gt_df1 = evaluate_mAA_from_model(
model_dir1, gt_csv,
threshold_csv,
dataset=ds1, scene=sc1, verbose=True
 )
print("mAA:",mAA1)




[eval] Similarity (max threshold) transform:
 [[ 0.84558241  0.12501454 -0.86572881  0.30679416]
 [-0.06760999  1.20985295  0.10867074  0.15638522]
 [ 0.87209168 -0.02741908  0.84783779  0.77256832]
 [ 0.          0.          0.          1.        ]]

[mAA] dataset=ETs scene=ET => 8.33%
mAA: 0.08333333333333333

[eval] Similarity (max threshold) transform:
 [[ 1.19760937 -0.0112851   0.11114377  0.30589692]
 [ 0.00518928  1.20098376  0.066027    0.2160948 ]
 [-0.11159463 -0.06526209  1.19584112  0.56190442]
 [ 0.          0.          0.          1.        ]]

[mAA] dataset=ETs scene=another_ET => 11.90%
mAA: 0.11904761904761904
